# Activity 4.1 - Cleaning Walmart Data the OpenRefine Way

In this activity, you will practice what you learned in Lecture 4.5 by cleaning up a data set containing information on various Walmart locations.

In [2]:
import pandas as pd
from dfply import *

#### Initial Tasks

1. Try to read in the `./data/Walmart_United_States_&_Canada.csv` file and verify that you get an encoding error.  This means that the [character encoding](https://en.wikipedia.org/wiki/Character_encoding) isn't the default of `utf-8`.  The easiest way to fix this is to open and save the file in Visual Studio Code.

In [22]:
walmart = pd.read_csv('./data/Walmart_United_States_&_Canada.csv', names = ['Lat','Lon','Store','Address'])
walmart.head()

,Lat,Lon,Store,Address
0,-114.005671,51.262567,"Walmart Supercentre; #1050,","2881 Main St SW,Airdrie ,AB T4B 3G5,(403) 945-..."
1,-111.900542,50.577939,"Walmart Supercentre; #3658,","917 3rd St W,Brooks ,AB T1R 1L5,(403) 793-2111"
2,-114.039133,51.107253,"Walmart Supercentre; #3013,","1110 57th Ave NE,Calgary ,(NOP),AB T2E 9B7,(40..."
3,-114.138488,51.040871,"Walmart Supercentre; #3009,Gas,","1212 37 St SW,Calgary ,(NOP),AB T3C 1S3,(403) ..."
4,-114.028603,50.930551,"Walmart; #1144,","1221 Canyon Meadows Dr SE,Calgary ,AB T2J 6G2,..."


2. Read in the data to verify that the encoding is fixed, but that there are two more problems.  What are they?

<font color="blue"> Columns we're comined, rows reading as headers </font>

3. Take another look at the file in VS Code and determine solutions to the two/three issues, then read in the data correctly by passing `pd.read_csv` the correct defaults for this data. **Note.** Leave the `"` in place for now, as they serve an important role here!

## Cleaning up the store information.

As hinted at above, the presence of the `"` meant the two of the columns--one containing the store type/number and the other contain the address/phone number--are combined together.  This was done because some of these entries have a different number of variables.  For example, the store type/number column sometimes occasionally `Gas`.

In this part of the activity, you should apply the iterative OpenRefine approach to separate the information in the store column.

**Warning!** There is one entry that doesn't follow the same pattern as the rest.  You won't find this entry unless you carefully define/fix/eliminate patterns.

In [49]:
!pip install more_itertools

     |████████████████████████████████| 52 kB 2.1 MB/s eta 0:00:01


In [50]:
from more_dfply import case_when, ifelse
from more_dfply.facets import text_facet, text_filter


In [57]:
(walmart
>> select(X.Store)
>> filter_by(text_filter(X.Store, 'Gas'))
>> filter_by(text_filter(X.Store, 'Gas/Diesel'))
>> head(25))

,Store
70,"Murphy: USA; #7687,Gas/Diesel,"
71,"Walmart Supercenter; #0423,Gas/Diesel,"
75,"Walmart Supercenter; #1091,Gas/Diesel,"
76,"Murphy: USA; #7394,Gas/Diesel,"
79,"Walmart Supercenter; #0306,Gas/Diesel,"
80,"Murphy: USA; #6890,Gas/Diesel,"
89,"Sam's Club; #4989,Gas/Diesel,"
90,"Murphy: USA; #6735,Gas/Diesel,"
91,"Walmart Supercenter; #2739,Gas/Diesel,"
92,"Walmart Supercenter; #0764,Gas/Diesel,"


In [62]:
(walmart
>> select(X.Store)
>> mutate(fuel = case_when((text_filter(X.Store, 'Gas/Diesel'),
                                                "Gas/Diesel"),
                          (text_filter(X.Store, 'Gas'),
                                                "Gas"),
                                                (True, "None")
                                                ),
          Store = X.Store.replace('Gas(/Diesel)?,', '', regex = True)
)
)

,Store,fuel
0,"Walmart Supercentre; #1050,",None
1,"Walmart Supercentre; #3658,",None
2,"Walmart Supercentre; #3013,",None
3,"Walmart Supercentre; #3009,",Gas
4,"Walmart; #1144,",None
...,...,...
6811,"Walmart Supercenter; #4471,",Gas
6812,"Walmart Supercenter; #1457,",None
6813,"Walmart Supercenter; #1461,",None
6814,"Walmart Supercenter; #1508,",None


In [76]:
(walmart
>> select(X.Store)
>> filter_by(text_filter(X.Store, 'Gas/Diesel'))
>> filter_by(text_filter(X.Store, 'Gas'))
>> mutate(Store = X.Store.replace('Gas(/Diesel)?,', '', regex=True))
>> filter_by(~text_filter(X.Store, 'Walmart( Supercentre| Supercentre)?; #\d{4},', regex=True))
>> mutate(Store = X.Store.replace(r'Wm (.*)', r'Walmart \1', regex=True)))

/Users/paytonsimmons/.pyenv/versions/anaconda3-2022.05/lib/python3.9/site-packages/more_dfply/facets.py:30: UserWarning: This pattern is interpreted as a regular expression, and has match groups. To actually get the groups, use str.extract.
  return col.str.contains(pattern, case=case, regex=regex, na=na)


,Store
70,"Murphy: USA; #7687,"
71,"Walmart Supercenter; #0423,"
75,"Walmart Supercenter; #1091,"
76,"Murphy: USA; #7394,"
79,"Walmart Supercenter; #0306,"
...,...
6750,"Sam's Club; #6535,"
6772,"Walmart Supercenter; #2474,"
6782,"Sam's Club; #4936,"
6790,"Sam's Club; #6457,"


In [71]:
walmart['Store'].str.cat.(walmart[['Store Type', 'Store Number']], sep)

,Lat,Lon,Store,Address,Store Type,Store Number
0,-114.005671,51.262567,"Walmart Supercentre; #1050,","2881 Main St SW,Airdrie ,AB T4B 3G5,(403) 945-...",Walmart Supercentre,"#1050,"
1,-111.900542,50.577939,"Walmart Supercentre; #3658,","917 3rd St W,Brooks ,AB T1R 1L5,(403) 793-2111",Walmart Supercentre,"#3658,"
2,-114.039133,51.107253,"Walmart Supercentre; #3013,","1110 57th Ave NE,Calgary ,(NOP),AB T2E 9B7,(40...",Walmart Supercentre,"#3013,"
3,-114.138488,51.040871,"Walmart Supercentre; #3009,Gas,","1212 37 St SW,Calgary ,(NOP),AB T3C 1S3,(403) ...",Walmart Supercentre,"#3009,Gas,"
4,-114.028603,50.930551,"Walmart; #1144,","1221 Canyon Meadows Dr SE,Calgary ,AB T2J 6G2,...",Walmart,"#1144,"
